In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import os
from model import *
from typing import Tuple
from misc_utils import copy_model, test, train
import matplotlib.pyplot as plt

96


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
device

device(type='cuda')

In [3]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [4]:
print('==> Building model..')
args = {'q_a': 0, 
        'q_w': 0, 
        'quant_three_sig': False,
        'debug': False}
net = QuantisedMobileNetV2(**args)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

==> Building model..
Model Initialisation:
Quantising activations to 0 bits
Quantising weights to 0 bits
Quantising to three-sigma range: False, debug enabled: False


In [5]:
epochs = 4
learning_rate = 0.1
weight_decay = 4e-5
lr_schedule = [150, 250]
lr_decay = 0.1
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate,
                    momentum=0.9, weight_decay=weight_decay)

for epoch in range(epochs):
    train(net, criterion, optimizer, epoch, trainloader, device, lr_decay, lr_schedule)


Epoch: 0 and LR: 0.1000


RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 11.91 GiB total capacity; 1.29 GiB already allocated; 20.69 MiB free; 1.31 GiB reserved in total by PyTorch)

In [ ]:
if (res := test(net, criterion, testloader, device, 
                save_best=False, epoch=1, best_results=[best_acc, best_epoch], 
                save_model_path='best_net.pth')) is not None: [best_acc, best_epoch] = res
print(best_acc)